In [ ]:
# Third-party imports
%matplotlib inline
import mxnet as mx
from mxnet import gluon
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

# Download and read data

In [ ]:
from gjopen.utilities import df_tools, read_files 
df = read_files.download_and_read_noaa_txt("ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_trend_gl.txt")
data = df_tools.set_noaa_datetime_index(df)

In [ ]:
data

# build dataset

In [ ]:

import mxnet as mx

ctx = mx.Context("gpu")

In [ ]:
mx.context.num_gpus()

In [ ]:
freq="MS"
data = data.resample(freq).apply(lambda ser: ser.iloc[-1,])
from gluonts.dataset.common import ListDataset
training_data = ListDataset(
    [{"start": data.index[0], "target": data.cycle[:"2018-01-31 00:00:00"]}], # TODO: index in programatical way
    freq = freq
)
validation_data = ListDataset(
    [{"start": data.index[0], "target": data.cycle[:"2019-01-31 00:00:00"]}], # TODO: index in programatical way
    freq = freq
)
test_data = ListDataset(
    [{"start": data.index[0], "target": data.cycle}],
    freq = freq
)

In [ ]:

from gluonts.model.deepar import DeepAREstimator
from gluonts.trainer import Trainer

estimator = DeepAREstimator(
    freq=freq, 
    prediction_length=24, 
    trainer=Trainer(epochs=100, ctx = ctx),
)
predictor = estimator.train(
    training_data=training_data,
    validation_data=validation_data
)



In [ ]:

from gluonts.dataset.util import to_pandas

for test_entry, forecast in zip(test_data, predictor.predict(test_data)):
    to_pandas(test_entry)[-60:].plot(linewidth=2)
    forecast.plot(color='g', prediction_intervals=[50.0, 90.0])
plt.grid(which='both')

In [ ]:
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.trainer import Trainer

In [ ]:
estimator1 = SimpleFeedForwardEstimator(
    num_hidden_dimensions=[100],
    prediction_length=24,
    context_length=100,
    freq=freq,
    trainer=Trainer(ctx="cpu", 
                    epochs=100, 
                    learning_rate=1e-3, 
                    num_batches_per_epoch=100
                   )
)

In [ ]:
predictor1 = estimator1.train(training_data)

In [ ]:

from gluonts.dataset.util import to_pandas

for test_entry, forecast in zip(test_data, predictor1.predict(test_data)):
    to_pandas(test_entry)[-60:].plot(linewidth=2)
    forecast.plot(color='g', prediction_intervals=[50.0, 90.0])
plt.grid(which='both')